# Integrating tools/function with LLM using Azure OpenAI

In [1]:
!pip install openai --quiet

In [3]:
# Define constants for the Azure OpenAI API configuration
api_key = "xxxxxxxxxxxxxx"
api_version = "2023-07-01-preview" # "2023-05-15"
azure_endpoint = "https://xxxxxxxxx.openai.azure.com/"
model_name = "gpt-35-turbo"

In [4]:
import os
from openai import AzureOpenAI
import json
import requests

client = AzureOpenAI(
  azure_endpoint = azure_endpoint, 
  api_key=api_key,  
  api_version=api_version
)


In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. The Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code.

## Supported models
Not all model versions are trained with function calling data. Function calling is supported with the following models: gpt-4, gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-4-0613, gpt-3.5-turbo, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, and gpt-3.5-turbo-0613

In addition, parallel function calls is supported on the following models: gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-3.5-turbo-0125, and gpt-3.5-turbo-1106

In [5]:
# Define the OpenWeatherMap API key
OWM_API_KEY = "29af1cea50a401d8e624eea4660b3f59"

def get_current_weather(location, unit="kelvin"):
    """
    Fetches the current weather information for a given location.

    Parameters:
    - location: str, the name of the location (e.g., "Paris").
    - unit: str, the unit of temperature (default is "kelvin").

    Returns:
    - str: JSON formatted string containing weather information.
    """
    # Construct the API request URL
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={OWM_API_KEY}"
    
    # Send the API request
    response = requests.get(url)
    
    # Parse the temperature and weather forecast from the response
    temp = response.json()['main']['temp']
    forecast = [response.json()['weather'][0]['main'], response.json()['weather'][0]['description']]

    # Create a dictionary with the weather information
    weather_info = {
        "location": location,
        "temperature": temp,
        "unit": 'Kelvin',
        "forecast": forecast
    }
    
    # Return the weather information as a JSON string
    return json.dumps(weather_info)

In [6]:
get_current_weather("Paris")

'{"location": "Paris", "temperature": 290.43, "unit": "Kelvin", "forecast": ["Rain", "light rain"]}'

In [7]:
get_current_weather("Manila")

'{"location": "Manila", "temperature": 304.47, "unit": "Kelvin", "forecast": ["Clouds", "overcast clouds"]}'

In [8]:
messages = [{"role":"user",'content':"what is AI?"}]
results = client.chat.completions.create(model = model_name, messages = messages)
print(results.model_dump_json(indent=2))

{
  "id": "chatcmpl-9bNSMJQblXNxi55gX0f1pLPk5l7h3",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "AI stands for Artificial Intelligence. It refers to the simulation of human intelligence in machines that are programmed to perform tasks that would normally require human intelligence, such as visual perception, speech recognition, problem-solving, learning, and decision-making. AI enables machines to process and analyze large amounts of data, identify patterns, and make autonomous decisions or predictions. It utilizes various techniques like machine learning, neural networks, natural language processing, and computer vision to mimic human intelligence.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "f

In [9]:

# Define tools/functions for the LLM to use
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit","kelvin"]},
                    },
                    "required": ["location"],
                },
            },
        },   
    ]

In [17]:
def get_response(messages, tools, model=model_name):
    """
    Handles interaction with the LLM, including making function calls if needed.

    Parameters:
    - messages: list, the conversation history.
    - tools: list, the functions/tools available to the LLM.
    - model: str, the model name to use (default is model_name).

    Returns:
    - str: The response from the LLM or function.
    """
    # Create a chat completion with the given messages and tools
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice='auto',
        temperature=0.2,
    )
    #print(response.model_dump_json(indent=2))
    # Get the generated response and tool calls (if any)
    response = response.choices[0].message
    tool_calls = response.tool_calls

    try:
        if tool_calls:
            print("Making a function call")
            # Available functions
            available_functions = {
                "get_current_weather": get_current_weather,
            }
            print(tool_calls)

            # Extend conversation with assistant's reply
            messages.append(response)

            # Handle each function call
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

            # Get a new response from the model with the function response
            second_response = client.chat.completions.create(
                model=model_name,
                messages=messages,
            )
            return second_response
        else:
            return response.content
    except Exception as e:
        print("Error occurred", e)
        return response

In [18]:
# Example usage of get_response function
messages = [{"role": "user", "content": "Provide a 2 line explanation for AI"}]
response = get_response(messages, tools)
print(response)

{
  "id": "chatcmpl-9bNr00U46cFQDLker3zxluTwSIvHf",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "AI, or Artificial Intelligence, refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. It involves the development of computer systems that can perform tasks that would typically require human intelligence, such as speech recognition, problem-solving, and decision-making.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
      

In [19]:
messages = [{"role": "user", "content": "How is the weather in Tokyo today?"}]
response = get_response(messages, tools)
print(response.choices[0].message.content)

{
  "id": "chatcmpl-9bNrC4vy2ReOwPc1QPfNQh6sChtRb",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_FfE6kse2GSyJL63Fqb9nWR0Z",
            "function": {
              "arguments": "{\n  \"location\": \"Tokyo\"\n}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      },
      "content_filter_results": {}
    }
  ],
  "created": 1718696870,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 87,
    "total_tokens": 104
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "sel